# Trabalho Prático 1 - Compressão de imagens
## Introdução à Computação Visual - 2020/1
#### Integrantes:
* Otávio Augusto Silva - 2016006808
* Luiz Henrique de Melo Santos - 2017014464

Neste Trabalho Prático será feita a implementação de uma aplicação para compressão de imagens sem perdas, por meio do Algoritmo de Huffman para a codificação da imagem.

In [23]:
import numpy
import warnings
from itertools import chain
from collections import Counter
from PIL import Image, ImageChops

Para a realizaçãp da compactação (e descompressão) com eficácia, precisamos ser capazes de tratar nossa saída (ou entrada) como um fluxo de bits individuais. As bibliotecas Python padrão não fornecem uma solução direta para fazer isso - na granularidade mais baixa, só podemos ler ou gravar um arquivo um byte de cada vez.

Como queremos codificar valores que podem consistir em vários bits, é essencial decodificar como eles devem ser ordenados com base na significância. Por isto ordenamos do mais significativo para o menos significativo, por meio de um BitStream.

Para os bits de entrada, queremos ler 1 ou mais bits por vez. Para isso, carregamos os bytes do arquivo, convertemos cada byte em uma lista de bits e o adicionamos ao buffer, até que haja o suficiente para atender à solicitação de leitura.

O comando flush, neste caso, limpa o buffer (garantindo que ele contenha apenas zeros).

In [24]:
# Classe responsavel pelo armazenamento e operacoes na estrutura de bits que
# sera utilizada na aplicacao para o arquivo de entrada

class InputBitStream:
    def __init__(self, file_name): 
        self.file_name = file_name
        self.file = open(self.file_name, 'rb') 
        self.bytes_read = 0
        self.buffer = []

    def read_bit(self):
        return self.read_bits(1)[0]

    def read_bits(self, count):
        while len(self.buffer) < count:
            self._load_byte()
        result = self.buffer[:count]
        self.buffer[:] = self.buffer[count:]
        return result

    def flush(self):
        assert(not any(self.buffer))
        self.buffer[:] = []

    def _load_byte(self):
        value = ord(self.file.read(1))
        self.buffer += pad_bits(to_binary_list(value), 8)
        self.bytes_read += 1

    def close(self): 
        self.file.close()

Uma vez que a API de entrada/saída do arquivo permite salvar apenas bytes inteiros, foi criada uma classe 'wrapper' que armazenará os bits gravados em um fluxo na memória.

Foram criados meios para escrever um único bit, bem como uma sequência de bits.

Cada comando de gravação (de 1 ou mais bits) adicionará primeiro os bits ao buffer. Uma vez que o buffer contém mais de 8 bits, grupos de 8 bits são removidos da frente, convertidos em um inteiro no intervalo [0-255] e salvos no arquivo de saída. Isso é feito até que o buffer contenha menos de 8 bits.

Finalmente, foi fornecida uma maneira de "liberar" o fluxo - quando o buffer não está vazio, porém não contém bits suficientes para fazer um byte inteiro, adicione zeros à posição menos significativa até que haja 8 bits e, em seguida, escreva o byte. Foi necessário isto quando estamos fechando o fluxo de bits (e há alguns outros benefícios que veremos mais tarde).

In [25]:
# Classe responsavel pelo armazenamento e operacoes na estrutura de bits que
# sera utilizada na aplicacao para o arquivo de saida

class OutputBitStream:
    def __init__(self, file_name): 
        self.file_name = file_name
        self.file = open(self.file_name, 'wb') 
        self.bytes_written = 0
        self.buffer = []

    def write_bit(self, value):
        self.write_bits([value])

    def write_bits(self, values):
        self.buffer += values
        while len(self.buffer) >= 8:
            self._save_byte()        

    def flush(self):
        if len(self.buffer) > 0: # Add trailing zeros to complete a byte and write it
            self.buffer += [0] * (8 - len(self.buffer))
            self._save_byte()
        assert(len(self.buffer) == 0)

    def _save_byte(self):
        bits = self.buffer[:8]
        self.buffer[:] = self.buffer[8:]

        byte_value = from_binary_list(bits)
        self.file.write(bytes([byte_value]))
        self.bytes_written += 1

    def close(self): 
        self.flush()
        self.file.close()

Funções auxiliares que serao utilizadas para manipulacoes na arvore, na compressao e na descompressao da imagem.

In [26]:
# Retorna o tamanho da imagem a partir do arquivo binario

def raw_size(width, height):
    header_size = 2 * 16  # Altura e largura como 16 valores de bits
    pixels_size = 3 * 8 * width * height  # 3 canais, sendo 8 bits por canal
    return (header_size + pixels_size) / 8

In [27]:
# Analisa duas imagens sao iguais

def images_equal(file_name_a, file_name_b):
    image_a = Image.open(file_name_a)
    image_b = Image.open(file_name_b)

    diff = ImageChops.difference(image_a, image_b)

    return diff.getbbox() is None

In [28]:
# Determina o PSNR entre duas imagens

def PSNR(X1, X2, max_pixel=255):
    mse = numpy.mean((X1 - X2)**2)
    if mse > 0:
        return 20 * numpy.log10(max_pixel / mse ** .5)
    else:
        raise ValueError("MSE is 0, images are equal.")

In [29]:
# Promove uma contagem do numero de simbolos de uma determinada imagem

def count_symbols(image):
    pixels = image.getdata()
    values = chain.from_iterable(pixels)
    counts = Counter(values).items()
    return sorted(counts, key=lambda x: x[::-1])

In [30]:
# Promove a contrucao da arvore necessaria para a execucao do Algoritmo de Huffman

def build_tree(counts):
    nodes = [entry[::-1] for entry in counts]  # Criacao de tupla reversa (simbolo, contador)
    while len(nodes) > 1 :
        leastTwo = tuple(nodes[0:2])  # Combina os dois
        theRest = nodes[2:] # all the others
        combFreq = leastTwo[0][0] + leastTwo[1][0]  # Determina a frequencia dos pontos nas ramificacoes
        nodes = theRest + [(combFreq, leastTwo)]  # Adiciona um novo ponto de ramificacao ao final
        nodes.sort(key=lambda t: t[0])  # Ordena em um determinado lugar
    return nodes[0]  # Retorna a unica arvore resultante da lista

In [31]:
# Promove uma poda na arvore

def trim_tree(tree):
    p = tree[1]  # Ignora a frequencia de contagem em [0]
    if type(p) is tuple:  # Em cada no, corta para a esquerda, depois para a direita, e depois recombina
        return (trim_tree(p[0]), trim_tree(p[1]))
    return p  # Retorna o no

In [32]:
# Promove uma atribuicao de codigos para as folhas da arvore

def assign_codes_impl(codes, node, pat):
    if type(node) == tuple:
        assign_codes_impl(codes, node[0], pat + [0]) # Ponto de ramificacao - cria o ramo esquerdo
        assign_codes_impl(codes, node[1], pat + [1]) # Cria o ramo direito
    else:
        codes[node] = pat  # Uma folha, cria um codigo

In [33]:
# Promove uma atribuicao de codigos para a arvore

def assign_codes(tree):
    codes = {}
    assign_codes_impl(codes, tree, [])
    return codes

In [34]:
# Converte um inteiro para uma lista de bits

def to_binary_list(n):
    return [n] if (n <= 1) else to_binary_list(n >> 1) + [n & 1]

In [35]:
# Converte uma lista de bits para um numero inteiro

def from_binary_list(bits):
    result = 0
    for bit in bits:
        result = (result << 1) | bit
    return result

In [36]:
# Cria uma lista de prefixos de bits com zeros suficientes para atingir n digitos

def pad_bits(bits, n):
    assert(n >= len(bits))
    return ([0] * (n - len(bits)) + bits)

In [37]:
# Promove uma compressao da imagem original

def compressed_size(counts, codes):
    header_size = 2 * 16  # Altura e largura como valores para 16 bits

    tree_size = len(counts) * (1 + 8)  # Folhas: bandeira de 1 bit, simbolo de 8 bits cada
    tree_size += len(counts) - 1  # Nós: sinalizador de 1 bit cada
    if tree_size % 8 > 0:  # Padding para o próximo byte completo
        tree_size += 8 - (tree_size % 8)

    # Soma para cada símbolo de contagem * comprimento do código
    pixels_size = sum([count * len(codes[symbol]) for symbol, count in counts])
    if pixels_size % 8 > 0:  # Padding para o próximo byte completo
        pixels_size += 8 - (pixels_size % 8)

    return (header_size + tree_size + pixels_size) / 8

In [38]:
# Criacao do cabecalho para a codificacao da imagem original

def encode_header(image, bitstream):
    height_bits = pad_bits(to_binary_list(image.height), 16)
    bitstream.write_bits(height_bits)    
    width_bits = pad_bits(to_binary_list(image.width), 16)
    bitstream.write_bits(width_bits)

In [39]:
# Promove a codigicacao de Huffman por meio da arvore criada

def encode_tree(tree, bitstream):
    if type(tree) == tuple:  # Escreve 0 e codifica os filhos
        bitstream.write_bit(0)
        encode_tree(tree[0], bitstream)
        encode_tree(tree[1], bitstream)
    else:  # Folha - escreve 1, seguido pelo símbolo de 8 bits
        bitstream.write_bit(1)
        symbol_bits = pad_bits(to_binary_list(tree), 8)
        bitstream.write_bits(symbol_bits)

In [40]:
# Promove a codificacao de cada um dos pixels da imagem

def encode_pixels(image, codes, bitstream):
    for pixel in image.getdata():
        for value in pixel:
            bitstream.write_bits(codes[value])

A definição do formato do fluxo de bits compactado - existem três blocos de informações essenciais que são necessários para decodificar a imagem:

* A forma da imagem (altura e largura), supondo que seja uma imagem RGB de 3 canais.
* Informações necessárias para reconstruir os códigos de Huffman no lado da decodificação
* Dados de pixel codificados por Huffman

O formato compactado foi feito da seguinte maneira:

* Cabeçalho: altura da imagem (16 bits, sem sinal) / largura da imagem (16 bits, sem sinal).
* Tabela Huffman (começando alinhado com byte inteiro): usamos o tópico deste [link](https://stackoverflow.com/questions/759707/efficient-way-of-storing-huffman-tree/759766#759766) como apoio.
* Códigos de pixel (começando alinhado com byte inteiro): width * height * 3 Códigos Huffman em sequência

In [41]:
# Promove a compressao de uma determinada imagem por meio da codificacao de Huffman

def compress_image(in_file_name, out_file_name):
    print('Comprimindo "%s" -> "%s"' % (in_file_name, out_file_name))
    image = Image.open(in_file_name)
    print('Dimensoes da imagem: (altura=%d, largura=%d)' % (image.height, image.width))
    size_raw = raw_size(image.height, image.width)
    print('Tamanho da imagem RAW: %d bytes' % size_raw)
    counts = count_symbols(image)
    print('Contadores: %s' % counts)
    tree = build_tree(counts)
    print('Arvore: %s' % str(tree))
    trimmed_tree = trim_tree(tree)
    print('Arvore podada: %s' % str(trimmed_tree))
    codes = assign_codes(trimmed_tree)
    print('Codigos: %s' % codes)

    size_estimate = compressed_size(counts, codes)
    print('Tamanho estimado: %d bytes' % size_estimate)

    print('Escrevendo...')
    stream = OutputBitStream(out_file_name)
    print('* Offset do cabecalho: %d' % stream.bytes_written)
    encode_header(image, stream)
    stream.flush()  # O próximo bloco deve estar alinhado por byte
    print('* Offset da arvore: %d' % stream.bytes_written)
    encode_tree(trimmed_tree, stream)
    stream.flush()  # O próximo bloco deve estar alinhado por byte
    print('* Offset dos pixels: %d' % stream.bytes_written)
    encode_pixels(image, codes, stream)
    stream.close()

    size_real = stream.bytes_written
    print('Escritos %d bytes.' % size_real)

    print('Estaimativa de %scorrect.' % ('' if size_estimate == size_real else 'in'))
    print('Taxa de compressao: %0.2f' % (float(size_raw) / size_real))

In [42]:
# Promove a decodificacao do arquivo comprimido de uma imagem

def decode_header(bitstream):
    height = from_binary_list(bitstream.read_bits(16))
    width = from_binary_list(bitstream.read_bits(16))
    return (height, width)

In [43]:
# Promve a decodificacao da arvore gerada a partir do arquivo codificado

def decode_tree(bitstream):
    flag = bitstream.read_bits(1)[0]
    if flag == 1:  # Folha, símbolo de leitura e retorno
        return from_binary_list(bitstream.read_bits(8))
    left = decode_tree(bitstream)
    right = decode_tree(bitstream)
    return (left, right)

In [44]:
# Promove a decodificacao de cada um dos valores encontrados

def decode_value(tree, bitstream):
    bit = bitstream.read_bits(1)[0]
    node = tree[bit]
    if type(node) == tuple:
        return decode_value(node, bitstream)
    return node

In [45]:
# Promove a decodificacao de cada um dos pixels presentes

def decode_pixels(height, width, tree, bitstream):
    pixels = bytearray()
    for i in range(height * width * 3):
        pixels.append(decode_value(tree, bitstream))
    return Image.frombytes('RGB', (width, height), bytes(pixels))

In [51]:
# Promove a descompressao de uma determinada imagem codificada por Huffman e comprimida

def decompress_image(in_file_name, out_file_name):
    print('Descomprimindo "%s" -> "%s"' % (in_file_name, out_file_name))

    print('Lendo...')
    stream = InputBitStream(in_file_name)
    print('* Offset do cabecalho: %d' % stream.bytes_read)
    height, width = decode_header(stream)
    stream.flush()  # O próximo bloco deve estar alinhado por byte
    print('* Offset da arvore: %d' % stream.bytes_read)    
    trimmed_tree = decode_tree(stream)
    stream.flush()  # O próximo bloco deve estar alinhado por byte
    print('* Offset do pixel: %d' % stream.bytes_read)
    image = decode_pixels(height, width, trimmed_tree, stream)
    stream.close()
    print('Le %d bytes.' % stream.bytes_read)

    print('Tamanho da imagem: (altura=%d, largura=%d)' % (height, width))
    print('Arvore podada: %s' % str(trimmed_tree))
    image.save(out_file_name)

#### Escreva o nome do do arquivo da imagem que você deseja utilizar como entrada aqui:

In [55]:
FILENAME = "tiger.bmp"

Compressão da imagem.

In [56]:
compress_image(FILENAME, "code.bin")

Comprimindo "tiger.bmp" -> "code.bin"
Dimensoes da imagem: (altura=354, largura=630)
Tamanho da imagem RAW: 669064 bytes
Contadores: [(246, 243), (242, 254), (243, 256), (240, 257), (241, 272), (245, 285), (244, 290), (237, 294), (235, 295), (238, 300), (239, 301), (236, 305), (247, 306), (233, 307), (234, 313), (248, 316), (230, 329), (232, 340), (225, 351), (249, 355), (231, 360), (227, 363), (229, 363), (226, 364), (255, 367), (228, 373), (250, 381), (223, 391), (215, 405), (219, 406), (224, 406), (222, 407), (218, 415), (220, 427), (221, 429), (214, 434), (251, 436), (213, 452), (217, 460), (208, 464), (216, 467), (253, 468), (206, 481), (211, 481), (210, 484), (254, 490), (209, 491), (212, 494), (252, 501), (207, 503), (204, 512), (205, 518), (200, 530), (198, 534), (201, 554), (194, 559), (202, 561), (196, 563), (203, 572), (197, 574), (191, 580), (192, 584), (195, 587), (193, 596), (190, 597), (185, 609), (199, 622), (189, 628), (188, 631), (187, 633), (184, 650), (186, 655), (1

#### Descompressão da imagem.

In [57]:
decompress_image("code.bin", "out_image.png")

Descomprimindo "code.bin" -> "out_image.png"
Lendo...
* Offset do cabecalho: 0
* Offset da arvore: 4
* Offset do pixel: 324
Le 617757 bytes.
Tamanho da imagem: (altura=354, largura=630)
Arvore podada: (((((((20, 24), 9), (((108, ((217, 208), 166)), 22), (21, 18))), (((23, (107, (161, (216, 253)))), (25, 81)), ((82, 15), (((162, (206, 211)), 105), 16)))), ((((26, (((210, 254), 163), ((209, 212), 160))), (27, 80)), (((106, (((246, 242), 252), 156)), 28), (79, (104, (153, 159))))), (((14, (((207, 204), 155), 103)), (77, ((((243, 240), 205), 158), 102))), ((30, 29), (((154, 151), (157, (200, 198))), 78))))), (((((31, (101, (149, 152))), 8), ((13, 32), (((147, 145), ((201, (241, 245)), 148)), 76))), ((((100, 99), 34), (33, (((194, 202), (196, 203)), (150, 146)))), ((75, 74), ((97, 98), (((197, 191), 142), ((192, (244, 237)), (195, (235, 238)))))))), ((((35, ((139, 141), 96)), (12, (((193, 190), 143), (144, ((239, 236), 185))))), ((36, 73), (37, 71))), ((0, ((95, (((247, 233), 199), (189, (2

Compraração entre a imagem original e a imagem final descomprimida. Como é um método sem perdas, as imagem devem ser o mais iguais possíveis.

In [59]:
img1 = Image.open(FILENAME)
img2 = Image.open("out_image.png")
PSNR(numpy.array(img1), numpy.array(img2))

ValueError: MSE is 0, images are equal.